In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Quiz 1:

QUIZ QUESTION
Using Derek's example in the previous window, deconstruct the window function alias into its two parts: the alias part and the window function part.

Solution:

Alias part: WINDOW main_window AS

Window function part: (PARTITION BY account_id ORDER BY STRFTIME('%Y-%m',occurred_at))

Task 1:

Now, create and use an alias to shorten the following query (which is different than the one in Derek's previous video) that has multiple window functions. Name the alias account_year_window, which is more descriptive than main_window in the example above.

In [12]:
pd.read_sql_query(sql='''
SELECT id,
       account_id,
       standard_qty,
       STRFTIME('%Y', occurred_at) AS year,
       DENSE_RANK() OVER account_year_window AS dense_rank,
       RANK() OVER account_year_window AS rank,
       SUM(standard_qty) OVER account_year_window AS sum_std_qty,
       COUNT(standard_qty) OVER account_year_window AS count_std_qty,
       AVG(standard_qty) OVER account_year_window AS avg_std_qty,
       MIN(standard_qty) OVER account_year_window AS min_std_qty,
       MAX(standard_qty) OVER account_year_window AS max_std_qty
FROM orders
WINDOW account_year_window AS (PARTITION BY account_id ORDER BY STRFTIME('%Y',occurred_at));
''', con=conn)[:50]

,id,account_id,standard_qty,year,dense_rank,rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,1,1001,123,2015,1,1,1430,5,286.000000,85,526
1,2,1001,190,2015,1,1,1430,5,286.000000,85,526
2,3,1001,85,2015,1,1,1430,5,286.000000,85,526
3,4307,1001,506,2015,1,1,1430,5,286.000000,85,526
4,4308,1001,526,2015,1,1,1430,5,286.000000,85,526
5,4,1001,144,2016,2,6,7896,28,282.000000,85,566
6,5,1001,108,2016,2,6,7896,28,282.000000,85,566
7,6,1001,103,2016,2,6,7896,28,282.000000,85,566
8,7,1001,101,2016,2,6,7896,28,282.000000,85,566
9,8,1001,95,2016,2,6,7896,28,282.000000,85,566


Solution:

SELECT id,
       account_id,
       DATE_TRUNC('year',occurred_at) AS year,
       DENSE_RANK() OVER account_year_window AS dense_rank,
       total_amt_usd,
       SUM(total_amt_usd) OVER account_year_window AS sum_total_amt_usd,
       COUNT(total_amt_usd) OVER account_year_window AS count_total_amt_usd,
       AVG(total_amt_usd) OVER account_year_window AS avg_total_amt_usd,
       MIN(total_amt_usd) OVER account_year_window AS min_total_amt_usd,
       MAX(total_amt_usd) OVER account_year_window AS max_total_amt_usd
FROM orders 
WINDOW account_year_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))